In [1]:
import win32com.client
from datetime import date
import calendar
import os

In [2]:
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

main_folder = outlook.Folders("anhtri@radlawgroup.com")

inbox = main_folder.Folders("Inbox")

emails = inbox.Items


In [9]:
flagged_emails = []

for email in emails:
    try:
        if email.FlagStatus == 2:
            flagged_emails.append(email)
    except Exception as e:
        continue

restricted_emails = emails.Restrict(
    "[ReceivedTime] >= '"
    + "08/01/2025"
    + "' AND [ReceivedTime] <= '"
    + "08/31/2025"
    + "'"
)

for email in restricted_emails:
    try:
        print(email.ReceivedTime, email.Subject)
    except Exception as e:
        continue


2025-08-06 09:44:15.015000+00:00 Case Management
2025-08-06 09:26:43.719000+00:00 RE: Pham v. Alqasieer-25-0045760 (Case No.: 25CV458273; Santa Clara County)
2025-08-06 09:26:42.320000+00:00 Jessie Victoria; DOL: 01/02/2025
2025-08-05 19:38:13.664000+00:00 FW: Tahanejad v. Mojabi; RF09446799 Alameda - Fee Agreement
2025-08-05 19:33:18.199000+00:00 RE: Tahanejad v. Mojabi RF09446799
2025-08-05 19:32:21.192000+00:00 RE: He now claims he can't resond until Thursday
2025-08-05 19:30:05.983000+00:00 FW: Tahanejad v. Mojabi RF09446799
2025-08-05 18:53:11.971000+00:00 Tahanejad v. Mojabi; RF09446799 Alameda
2025-08-05 18:02:22.321000+00:00 Re: Tahanejad v. Mojabi; RF09446799 Alameda - Fee Agreement
2025-08-05 17:58:48.352000+00:00 RE: He now claims he can't resond until Thursday
2025-08-05 17:50:20.798000+00:00 FW: Tahanejad v. Mojabi RF09446799
2025-08-05 17:46:16.587000+00:00 RE: Tahanejad v. Mojabi; RF09446799 Alameda - Fee Agreement
2025-08-05 17:25:05.497000+00:00 Recent activity in MyCa

In [99]:
start_of_month = date.today().replace(day=1)
_, num_days = calendar.monthrange(date.today().year, date.today().month)
end_of_month = date.today().replace(day=num_days)

flagged_emails_in_month = []

for email in flagged_emails:
    received_date = email.ReceivedTime.date()
    if received_date >= start_of_month and received_date <= end_of_month:
        flagged_emails_in_month.append(email)


In [ ]:

flagged_emails_folder_name = f"Flagged Emails {start_of_month.strftime("%m-%d-%y")} - {end_of_month.strftime("%m-%d-%y")}"

billing_path = os.path.join(os.path.expanduser("~"), "Desktop", "Billing", flagged_emails_folder_name) + ".pst"

# Only create the folder if it doesn't exist
if not os.path.exists(billing_path):
    new_pst = outlook.AddStore(billing_path)
else:
    print(f"Folder {flagged_emails_folder_name} already exists")

# Get the store with the path
flagged_emails_store = None

for store in outlook.Stores:
    if (store.FilePath == billing_path):
        flagged_emails_store = store

flagged_emails_root = flagged_emails_store.GetRootFolder()



Folder Flagged Emails 08-01-25 - 08-31-25 already exists
Outlook Data File


In [108]:
# Use a set of emails to avoid duplicates
existing_emails_in_store = set()

for email in flagged_emails_root.Items:
    existing_emails_in_store.add(email.Subject)

copy_count = 0

for flagged_email in flagged_emails:
    if flagged_email.Subject in existing_emails_in_store:
        print(f"Skipping {flagged_email.Subject}: Already exists!")
        continue

    print(f"Copying {flagged_email.Subject} to {flagged_emails_root.Name}")
    flagged_email.Copy().Move(flagged_emails_root)
    copy_count += 1

print(f"Copied {copy_count} emails to {flagged_emails_root.Name}")






Skipping FW: New Cases: Accident.com: Already exists!
Skipping Marriage of Heidari - Case No. 20FL001880 Inbox Mabel Lin: Already exists!
Skipping FW: Closing the loop: Already exists!
Skipping RE: Medical Records Summarization-  Only $ 0.16 per page  - FAST TURNAROUND TIME: Already exists!
Skipping Motorcycle Accident Webpage attached: Already exists!
Skipping FW: Billboards around California: Already exists!
Skipping RE: Two latest blog posts attached.  : Already exists!
Skipping Medical records summary: Already exists!
Skipping Santa Clara log in: Already exists!
Skipping RE: Taghavi v. Khoshkbari, Case No.: 23FL000295, Santa Clara County: Already exists!
Skipping Marriage of Heidari and Setayesh - Ex Parte RFO: Already exists!
Skipping Marriage of Heidari and Setayesh - Letter of Today's Date: Already exists!
Skipping RLG Billing Test: Already exists!
Skipping FW: Rad v. Mojabi, Case No.: HG17865400, Alameda County, Petitioner - Civil Harassment (June 12, 2025 Hearing at 9:00 a.m. 